# DraCor API: first steps

This short chapter shows how to list DraCor corpora and peek at a few plays via the HTTP API using a tiny helper (`utils/dracor_client.py`).

> **Learning goals**
> - Call the DraCor API from Python.
> - List available corpora and preview plays.
>
> **Requirements**
> - Basic Python; internet connection.
>
> **What you'll do**
> 1) Import a small API helper. 2) Fetch corpora. 3) Count plays in a few corpora.
>
> **Exercise (at the end)**
> - Repeat for different corpora; compare sizes and languages.

In [ ]:
# Bootstrap import path so `from utils import dracor_client` works everywhere
import os, sys
from pathlib import Path

# Try likely project roots relative to the notebook
_candidates = [
    Path.cwd(),                                 # running from docs/
    Path.cwd().parent,                          # running from project root
    Path(__file__).resolve().parents[1] if "__file__" in globals() else None,
]
for _p in [p for p in _candidates if p]:
    if (_p / "utils" / "dracor_client.py").exists():
        if str(_p) not in sys.path:
            sys.path.insert(0, str(_p))
        break

import pandas as pd
pd.set_option("display.max_colwidth", 80)
pd.set_option("display.precision", 0)

from datetime import datetime
print("Last run:", datetime.utcnow().strftime("%Y-%m-%d %H:%M UTC"))

In [ ]:
from utils import dracor_client as dc

corpora = dc.corpora()
[c["slug"] for c in corpora[:10]]

In [ ]:
import pandas as pd

df_corpora = pd.DataFrame(corpora)
keep = [c for c in ["id", "name", "slug", "description", "languages"] if c in df_corpora.columns]
df_corpora[keep].head()

## How many plays per corpus?

Let’s sample a handful of corpora and count how many plays each contains. If any corpus
responds with an error, we’ll skip it and keep going (useful for transient API hiccups).

In [ ]:
import pandas as pd
import requests

sample = corpora[:8]  # take the first few corpora for a quick preview

rows = []
failed = []

for c in sample:
    slug = c["slug"]
    try:
        plays = dc.corpus_plays(slug)
        rows.append({"corpus": slug, "n_plays": len(plays)})
    except requests.HTTPError as e:
        status = getattr(e.response, "status_code", "HTTP")
        failed.append({"corpus": slug, "error": f"{status}"})
    except Exception as e:
        failed.append({"corpus": slug, "error": str(e)})

df_counts = pd.DataFrame(rows).sort_values("n_plays", ascending=False).reset_index(drop=True)
df_counts

In [ ]:
import matplotlib.pyplot as plt

if not df_counts.empty:
    plt.figure(figsize=(6,3.5))
    plt.bar(df_counts["corpus"], df_counts["n_plays"])
    plt.title("Play counts (sampled corpora)")
    plt.ylabel("# plays")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.show()
else:
    print("No successful corpus responses to plot.")

## Exercise

1. Change the sampling to a custom set, e.g.
   ```python
   wanted = {'ger','rus','shake','dutch'}
   sample = [c for c in corpora if c['slug'] in wanted]
   ```
2. Re-run the counting cell. Which corpus has the most plays?
3. Optional: fetch the plays for one corpus (`dc.corpus_plays('ger')`) and list the first 5 titles as a table.

> **Takeaways**
> - You can access DraCor corpora and plays with a few lines of code.
> - Keeping results in tidy DataFrames makes quick summaries and plots trivial.
> - Next chapters will navigate plays, characters, and networks in more detail.